In [ ]:
#below is the feature i chose
# name
# price 
# porduct_url
# image_links
# color 
# description
# size 
# product_material 
# low_level [category of clothes: e.g. casual pants, dress] 
# scrapped_date

# these feature is not including:
# brand_name, not shown in the page
# gender (str) [men, women, or kids], this web only contain woman 
# secondhand (bool) [is it from a second hand retailer, already worn?] not shown in the page


In [13]:
# build two functions

#first get the data from the new_week_page
def get_products(url):
    # import packages
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(url)
    product_list = []
    if response.status_code != 200:
        return None
    try:    # set up beautifulsoup object
        results_page = BeautifulSoup(response.content,'lxml')
        all_products = results_page.find_all('div',{'class':"product-grid__item col-6 col-md-3"})
        for test in all_products:
            product_link = 'https://www.thereformation.com' + test.find('a',{'class' : "product-tile__anchor"}).get('href')
            product_name=test.find('div',{'class': "product-tile__body-section product-tile__name"}).get_text().strip('\n')
            product_price=test.find('span',{'class':'price--formated'}).get_text().strip('\n')
            product_image_links=[]
            for i in range(2): #only take two links of each product shown in the web
                product_image_links.append(test.find_all('img')[i].get('data-src'))
            product_colors, product_description,product_size,product_material,low_level = get_details(product_link)
            product_list.append((product_name,product_link,product_price,product_image_links,product_colors, product_description,product_size,product_material,low_level))
        return product_list
    except:
        return None
#second: get the details(color,size,material, category,description) of each product
def get_details(url):
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(url)
    if not response.status_code == 200:
        return None
    #get product color
    product_colors = []
    product_page = BeautifulSoup(response.content,'lxml')
    product = product_page.find('div',{'class':"pdp-main__details max-width--xmedium"})
    for j in product.find_all('span', {"class":"sr-only"}):
        color = j.get_text()[6:]
        product_colors.append(color)
    #get product description
    product_description = product.find('div', {"class":"cms-generic-copy"}).get_text().strip('\n')
    #get product size
    product_size = []
    for i in product.find_all('button', {"class":"product-attribute__anchor anchor--size selectable"}):
        product_size.append(i.get('data-attr-value'))
    
    #get product_material
    import re
    try:
        material = product.find('div', {"class":"pdp__accordion-content js-pdp-care"}).get_text().split('\n\n\n')[1]
        pattern = r'[0-9]+'
        product_material= material[re.search(pattern,material).span()[0]:].strip('.')
    except:
        product_material=''
    #get category 
    pattern2 = re.compile(r"var pageLoadGTMObj =(.*?);$", re.MULTILINE | re.DOTALL)
    script = product_page.find('script', text=pattern2).get_text()
    category = re.findall(r""".+?"category":"(.+?)" """,script,re.VERBOSE | re.DOTALL)
    category=category[0]

    return product_colors, product_description,product_size,product_material,category

In [14]:
url = 'https://www.thereformation.com/new/this-week'

pro =get_products(url)

In [15]:
# show all 
# pro

[('Pompano Linen Dress',
  'https://www.thereformation.com/products/pompano-linen-dress/1310476DNK.html?dwvar_1310476DNK_color=DNK',
  '$278.00',
  ['https://media.thereformation.com/image/list/fn_select:jq:first(.%5B%5D%7Cif%20has(%22metadata%22)%20then%20select(any(.metadata%5B%5D;%20.external_id%20==%20%22sfcc-is-main%22%20and%20.value.value%20==%20%22True%22))%20else%20empty%20end)/f_auto,q_auto,dpr_auto/w_900/1310476-DNK.json?_s=RAABAB0',
   'https://media.thereformation.com/image/list/fn_select:jq:first(.%5B%5D%7Cif%20has(%22metadata%22)%20then%20select(any(.metadata%5B%5D;%20.external_id%20==%20%22sfcc-gallery-position%22%20and%20.value%20==%202))%20else%20empty%20end)/f_auto,q_auto,dpr_auto/w_900/1310476-DNK.json?_s=RAABAB0'],
  ['Dandelion Check', 'White'],
  'Engineered for spinning around in an open field. The Pompano is a midi length dress with a sweetheart neckline and loose fitting, puff sleeves. It has a center front cutout and a smocked back bodice for a little extra st

In [16]:
import pandas as pd
productList = pd.DataFrame(pro,columns=['product name','product link','price','image links','colors','product_description','product_size','materials','category'])
productList

,product name,product link,price,image links,colors,product_description,product_size,materials,category
0,Pompano Linen Dress,https://www.thereformation.com/products/pompan...,$278.00,[https://media.thereformation.com/image/list/f...,"[Dandelion Check, White]",Engineered for spinning around in an open fiel...,"[000, 002, 004, 006, 008, 010, 012]",100% linen,Dresses
1,Jasmin Dress,https://www.thereformation.com/products/jasmin...,$218.00,[https://media.thereformation.com/image/list/f...,"[Citrus / Rhubarb, Parfait]","Silky silky. The Jasmin is a fitted, mini dres...","[000, 002, 004, 006, 008, 010, 012]",,Dresses
2,Agalia Two Piece,https://www.thereformation.com/products/agalia...,$148.00,[https://media.thereformation.com/image/list/f...,"[Deep Red, Warm Beige, Dark Blue]",Throw it on when you're running kind of late. ...,"[0XS, 00S, 00M, 00L, 0XL]",,Clothing
3,Orzo Linen Two Piece,https://www.thereformation.com/products/orzo-l...,$248.00,[https://media.thereformation.com/image/list/f...,"[Challah Check, Corvette, Malta, White]",Meant for each other. The Orzo is a two piece ...,"[000, 002, 004, 006, 008, 010, 012]",100% linen,Dresses
4,Lucila Linen Dress,https://www.thereformation.com/products/lucila...,$248.00,[https://media.thereformation.com/image/list/f...,[White],We think linen is kind of a big deal. The Luci...,"[000, 002, 004, 006, 008, 010, 012]",100% linen,Dresses
5,Isles Linen Dress,https://www.thereformation.com/products/isles-...,$248.00,[https://media.thereformation.com/image/list/f...,"[Flame, White]","Go forth and spring. The Isles is a strapless,...","[000, 002, 004, 006, 008, 010, 012]",100% linen,Dresses
6,Hilo Dress,https://www.thereformation.com/products/hilo-d...,$148.00,[https://media.thereformation.com/image/list/f...,"[Black, Palm Green]",Throw on and go. The Hilo is a midi dress with...,"[0XS, 00S, 00M, 00L, 0XL]","88% TENCEL™ lyocell, 12% spandex",Dresses
7,Cortese Top,https://www.thereformation.com/products/cortes...,$128.00,[https://media.thereformation.com/image/list/f...,"[Boheme, Campari, Saskia, Sofie]",We can't teach you how to tie a knot. The Cort...,"[000, 002, 004, 006, 008, 010, 012]","53% viscose, 47% rayon",Tops
8,Lizzi Linen Top,https://www.thereformation.com/products/lizzi-...,$128.00,[https://media.thereformation.com/image/list/f...,"[Citrus, White]","Romance for one. The Lizzi is a fitted, strapl...","[000, 002, 004, 006, 008, 010, 012]",100% linen,Tops
9,Liam Linen Pant,https://www.thereformation.com/products/liam-l...,$158.00,[https://media.thereformation.com/image/list/f...,"[Citrus, White]",Get some compliments. The Liam is a cropped le...,"[000, 002, 004, 006, 008]",100% linen,Pants & Shorts


In [17]:
pip install psycopg2-binary 

Note: you may need to restart the kernel to use updated packages.


In [18]:
import psycopg2.extras

In [ ]:
import psycopg2

In [ ]:
# I dont have a rellevant database, so below line I will not run
conn = psycopg2.connect(database = 'databasename', user = 'user', password = 'password', host = 'host', port = 'end')
curs=conn.cursor()
if len(productList) > 0:
    df_columns = list(productList)
    columns = ",".join(df_columns)
    values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 
    insert_stmt = "INSERT INTO {} ({}) {}".format('thereformation',columns,values)
    curs = conn.cursor()
    psycopg2.extras.execute_batch(curs, insert_stmt, productList.values)
    conn.commit()
    curs.close()